# Align Frames of Satellite image by Using Known TLE
Written by Kiyoaki Okudaira<br>
*Kyushu University Hanada Lab / University of Washington / IAU CPS SatHub<br>
(okudaira.kiyoaki.528@s.kyushu-u.ac.jp or kiyoaki@uw.edu)<br>
<br>
Align frames of satellite image (FITS) by SGP4 propagation with known TLE<br>
<br>
**History**<br>
coding 2026-01-27 : 1st coding<br>
<br>
(c) 2026 Kiyoaki Okudaira - Kyushu University Hanada Lab (SSDL) / University of Washington / IAU CPS SatHub

### Parameters
**Plate solve settings**<br>
Exept special cases, follow default / recommended values

In [ ]:
solve_images     = False# Plate solve images by astrometry.net | bool

solve_downsample = 2    # Downsample the image by factor (recommend 2) | int
solve_source_obj = 100  # Cut the source list to have this many items (recommend 10) | int
solve_cat_radius = 1    # Only search within radius of the field center [deg] (recommend 1) | int

# Advanced
solve_no_removel = True # NOT remove horizontal and vertical overdensities of sources | bool
solve_tagallstar = True # Grab all tag-along columns from index into RDLS file | bool
solve_over_write = True # Overwrite output files if they already exist | bool
solve_uniformize = 10   # Select sources uniformly (0=disable; default 10) | int

**TLE Download settings**<br>
Downloading from space-track.org

In [ ]:
norad_id = 12996 # NORAD CATALOG ID | int
tle_dt_range = 2 # TLE range from observation date | int > 0

### Import and initial settings
**PATH settings**

In [ ]:
basePATH = "/Users/kiyoaki/VScode/satphotometry_package"
# inputPATH = f"{basePATH}/input/images"
inputPATH = "/Users/kiyoaki/VScode/KUPT_Image/output/image"
spice_myfile_PATH = "/Users/kiyoaki/VScode/satphotometry_package/config/myfile.txt"

**Standard libraries**

In [ ]:
import os,shutil,json
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.wcs import WCS

**Satphotometry library**<br>
satphotometry.satorbit must be imported after starting up SPICE kernel

In [ ]:
from satphotometry import astrometry, gettle

**Custom functions**

In [ ]:
import cal_satorbit

**Observatory setting**

In [ ]:
from input.obs_site.KUPT import *
# obs_gd_lon_deg = 130.0 + (12.0 + 42.0 / 60.0) / 60.0    # [deg]
# obs_gd_lat_deg = 33.0 + (35.0 + 56.0 / 60.0) / 60.0     # [deg]
# obs_gd_height  = 0.073  # [km]

# wavelength_m = 0.5e-6   #[m]
# aperture_m   = 508.0e-3 #[m]

obs_params = [obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,wavelength_m,aperture_m]

**Space-track.org settings**<br>
Set your space-track.org identity and password at `./config/space-track-org.config`

In [ ]:
with open(f'{basePATH}/config/space-track-org.config') as f:
    st_config = json.load(f)
    st_user_id  = st_config["identity"] # space-track.org user id | str
    st_password = st_config["password"] # space-track.org password | str

**Warning handle**<br>
Ignore FITS header fixed warning and verify warning

In [ ]:
import warnings
from astropy.wcs.wcs import FITSFixedWarning
from astropy.io.fits.verify import VerifyWarning

warnings.simplefilter('ignore',FITSFixedWarning)
warnings.simplefilter('ignore',VerifyWarning)

### Plate solve
**List all input file names**

In [ ]:
inputfname_list = sorted([f for f in os.listdir(inputPATH) if (f.endswith('.fits') == True or f.endswith('.FITS') == True)])

**Astrometry.net options**<br>
Astrometry.net options should be written as dict format. Value should be `True` for option without any arguments.

In [ ]:
astrometry_option = {
    # solve settings
    "-L"            : "0.1",    # Lower bound of image scale estimate (--scale-low)
    "-H"            : "180.0",  # Upper bound of image scale estimate (--scale-high)
    "--objs"        : str(solve_source_obj),    # Cut the source list to have this many items
    "-z"            : str(solve_downsample),    # Downsample the image by factor int before running source extraction (--downsample)
    "--crpix-center": True,     # Set the WCS reference point to the image center
    "-9"            : solve_no_removel, # NOT remove horizontal and vertical overdensities of sources (--no-remove-lines)
    "--uniformize"  : str(solve_uniformize),    # Select sources uniformly using roughly this many boxes (0=disable; default 10)
    # output settings
    "-U"            : f'{basePATH}/tmp/astrometry/config/xylist',   # Output filename for xylist containing the image coordinate of stars from the index (--index-xyls)
    "-B"            : f'{basePATH}/tmp/astrometry/config/corr',     # Output filename for correspondences  (--corr)
    "--axy"         : f'{basePATH}/tmp/astrometry/config/axy',      # Output filename for augment xy list
    "-W"            : f'{basePATH}/tmp/astrometry/config/wcs',      # Output filename for WCS file (--wcs)
    "-R"            : f'{basePATH}/tmp/astrometry/config/rdls',     # Output filename for RDLS file (--rdls)
    "-M"            : f'{basePATH}/tmp/astrometry/config/match',    # Output filename for match file (--match)
    "-S"            : f'{basePATH}/tmp/astrometry/config/solved',   # Output file to mark that the solver succeeded (--solved)
    "-p"            : True,             # Don't create any plots of the results (--no-plots)
    "--tag-all"     : solve_tagallstar, # Grab all tag-along columns from index into RDLS file
    "-O"            : solve_over_write  # Overwrite output files if they already exist (--overwrite)
}

**Clean up temporary directory**

In [ ]:
if solve_images and solve_over_write is False:
    shutil.rmtree(f'{basePATH}/tmp/astrometry/config') if os.path.exists(f'{basePATH}/tmp/astrometry/config') else None
    shutil.rmtree(f'{basePATH}/tmp/astrometry/output') if os.path.exists(f'{basePATH}/tmp/astrometry/output') else None
    os.mkdir(f'{basePATH}/tmp/astrometry/config')
    os.mkdir(f'{basePATH}/tmp/astrometry/output')

**Plate solve**<br>
At the 1st frame: all world will be searched. At all other frames, given degree around 1st frame field will be solved.<br>
If plate solve failed, WCS information near the frame will be used instead.

In [ ]:
if solve_images:
    astrometry_result = None
    for input_fname in tqdm(inputfname_list):
        astrometry_option["-N"] = f"{basePATH}/tmp/astrometry/output/{input_fname}"
        
        if astrometry_result:
            img_header = fits.open(result_path)[0].header
            astrometry_option["-3"] = img_header["CRVAL1"]  # RA of field center for search [deg] or [hh:mm:ss] (-ra)
            astrometry_option["-4"] = img_header["CRVAL2"]  # DEC of field center for search [deg] or [dd:mm:ss] (-dec)
            astrometry_option["-5"] = str(solve_cat_radius) # Only search in indexes within 'radius' of the field center (--radius)

        # plate solve
        result_path,astrometry_result = astrometry.platesolve(f"{inputPATH}/{input_fname}",astrometry_option,True,True)

        # solve failure handling
        if astrometry_result is not True:
            org_img = fits.open(f"{inputPATH}/{input_fname}")[0]
            img_wcs_header = WCS(img_header).to_header(relax=True)
            for key in img_wcs_header:
                org_img.header[key] = img_wcs_header[key]
            fits.writeto(astrometry_option["-N"], org_img.data, org_img.header, output_verify="ignore", overwrite=True)

**Making solved file list**

In [ ]:
solvedfname_list = sorted([f for f in os.listdir(f"{basePATH}/tmp/astrometry/output") if (f.endswith('.fits') == True or f.endswith('.FITS') == True)]) if solve_images else inputfname_list
img_header = img_header if solve_images else fits.open(f"{inputPATH}/{solvedfname_list[0]}")[0].header

### Align Frames
**Download TLE file**<br>
Download latest Two-Line Element set from space-track.org

In [ ]:
status,tle_result = gettle.space_track.get_past_TLE(img_header["DATE-OBS"][0:10],tle_dt_range,norad_id,st_user_id,st_password)
if status == 200:
    tle_fname = "{0}_{1}".format(norad_id,gettle.parse_tle2epoch_fname(tle_result.splitlines()[1]))
    tle_PATH = f"{basePATH}/input/tle/{tle_fname}.tle"
    with open(tle_PATH,mode="w") as f:
        f.write(tle_result)
else:
    raise ValueError("Failed to download Two-Line Element set or NORAD catalog number is invalid.")
tle_result = tle_result.splitlines()

**SGP4 Propagation and align frames**

In [ ]:
for input_fname in tqdm(solvedfname_list):
    result_path = f"{basePATH}/tmp/astrometry/output/{input_fname}" if solve_images else f"{inputPATH}/{input_fname}"
    img_header = fits.open(result_path)[0].header
    img_data   = fits.open(result_path)[0].data
    img_wcs    = WCS(img_header)

    sum_header = img_header if input_fname == inputfname_list[0] else sum_header

    objname,norad_id,intldes,output = cal_satorbit.cal_satorbit(img_header["DATE-OBS"],img_header["DATE-END"],img_header["EXPTIME"],[tle_result[0],tle_result[1],tle_result[2]],obs_params,spice_myfile_PATH)

    orbit_xy = img_wcs.wcs_world2pix(np.array([[output[0]["ra[deg]"],output[0]["dec[deg]"]]]),1)[0]
    ref_xy   = orbit_xy if input_fname == inputfname_list[0] else ref_xy

    roll_xy  = ref_xy - orbit_xy
    rolled_data = np.roll(img_data, (roll_xy[1], roll_xy[0]), axis=(0, 1))

    sum_data = img_data if input_fname == inputfname_list[0] else sum_data + rolled_data

    fits.writeto(f"{basePATH}/output/images/{input_fname}", rolled_data, img_header, output_verify="ignore", overwrite=True)

**Save and display stacked image**

In [ ]:
fits.writeto(f"{basePATH}/output/images/sum.fits", sum_data, sum_header, output_verify="ignore", overwrite=True)

plt.figure(figsize=(10, 10))
plt.imshow(sum_data, cmap='gray', origin='lower', vmin=np.percentile(sum_data, 5), vmax=np.percentile(sum_data, 95))
plt.title("Stacked image")
plt.show()